In [22]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair
import data.scripts.gen_tables.search_best_inter as search_inter

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 
CSV_PATH = os.path.join(HOME, 'data/csv') 
GRAPH_PATH = os.path.join(HOME, 'data/graphs')

%config InlineBackend.figure_format ='retina'
plt.style.use('seaborn-talk')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [4]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
seq_file = os.path.join(CSV_PATH, 'seq-multi.csv')
seq_pkl = os.path.join(PKL_PATH, 'seq-multi.pkl')
%run $gen_seq --multi --csv $seq_file --output $seq_pkl

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq-multi.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd', 'ipc',
       'avg_dram_bw', 'avg_dram_eff', 'avg_row_locality', 'std_dram_bw',
       'ratio_dram_bw', 'MPKI', 'l2_access_density', '1_kidx', 'waves'],
      dtype='object')


In [5]:
col_seq = ['pair_str', '1_kidx', 'runtime',
           'ipc',
           'avg_dram_bw', 
          ]

draw_table(df_seq, col_seq).format({
    'runtime': '{:,}',
    'avg_dram_bw':'{:.4f}', 
    'sp_busy': '{:.2f}',
    'dp_busy': '{:.2f}',
    'int_busy': '{:.2f}',
    'tensor_busy': '{:.2f}',
    'sfu_busy': '{:.2f}',
})

pair_str,1_kidx,runtime,ipc,avg_dram_bw
nvd_conv-0,1,"190,346",2131.9,0.7242
nvd_conv-0,2,"208,164",1755.92,0.6698
parb_sad-0,1,"51,633",2686.45,0.0061
parb_sad-0,2,"21,556",203.199,0.4558
parb_sad-0,3,"12,806",89.9086,0.1249
parb_sad-1,1,"3,000,706",3754.08,0.1736
parb_sad-1,2,"2,324,970",153.001,0.5645
parb_sad-1,3,"541,865",172.562,0.5842


# Intra

In [8]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
intra_file = os.path.join(CSV_PATH, 'intra-multi.csv')
intra_pkl = os.path.join(PKL_PATH, 'intra-multi.pkl')

%run $gen_intra --out_intra $intra_pkl --seq $seq_pkl --csv $intra_file

df_intra = pd.read_pickle(intra_pkl)
df_intra.sort_values(['pair_str', '1_kidx'], inplace=True)
# col_intra = ['pair_str', '1_kidx', 'regs', 'thread_count', 'smem', 'intra']
# draw_table(df_intra, col_intra)

In [9]:
col_prod = ['pair_str_x', '1_kidx_x', 'pair_str_y', '1_kidx_y',
            'norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 
           'sum_comp', 'sum_dram',]
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')
pairs = df_seq.apply(lambda row: ':'.join([row['pair_str'], str(row['1_kidx'])]), axis=1)

In [10]:
app1 = widgets.Dropdown(
    options=pairs,
    value=pairs[0],
    rows=5,
    description='App 1:',
    disabled=False
)

app2 = widgets.Dropdown(
    options=pairs,
    value=pairs[0],
    rows=5,
    description='App 2:',
    disabled=False
)

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    
show_widgets()

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value --intra_pkl $intra_pkl --top
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
    print(df_prod.columns)
#     sns.lineplot('diff_mflat', 'sum_ipc', data=df_prod)
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    
button.on_click(onclick)

Dropdown(description='App 1:', index=6, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2…

Dropdown(description='App 2:', index=1, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2…

FloatSlider(value=0.75, continuous_update=False, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

Index(['level_0', 'index', 'pair_str_x', 'config_x', 'gpusim_version_x',
       'jobId_x', 'stall_icnt_to_l2_x', 'stall_l2_to_icnt_x',
       'stall_core_ldst_x', 'l1D_miss_rate_x',
       ...
       'cta_ratio_y', 'thread_ratio_y', 'smem_ratio_y', 'reg_ratio_y',
       'diff_mflat', 'sum_ipc', 'sum_comp', 'sum_dram', 'penalized', 'config'],
      dtype='object', length=105)


,pair_str_x,1_kidx_x,pair_str_y,1_kidx_y,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,sum_comp,sum_dram
0,parb_sad-1,2,nvd_conv-0,2,1.34946,1.08257,66,2.43204,2,10,0.228164,1.93421
1,parb_sad-1,2,nvd_conv-0,2,1.34946,1.16624,85,2.5157,2,8,0.243977,1.93639
2,parb_sad-1,2,nvd_conv-0,2,1.34946,1.14596,205,2.49542,2,6,0.238842,1.934
3,parb_sad-1,2,nvd_conv-0,2,1.27252,1.16624,858,2.43876,4,8,0.242732,1.93615


# CTX Ratio

In [103]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
ctx_csv = os.path.join(HOME, 'data/csv/ctx.csv')
output = os.path.join(PKL_PATH, 'pair_ctx.pkl')

print(gen_pair)

/home/serinatan/project/GPU-Virtualization-Benchmarks/util/data/scripts/gen_tables/gen_table_pair.py


In [104]:
%run $gen_pair --csv $ctx_csv --output $output --seq_pkl $seq_pkl --qos 0.5 --multi --how ctx

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [105]:
df_ctx= pd.read_pickle(output)

col_ctx = ['1_bench', '2_bench', '1_ctx', '2_ctx', 'runtime', 'norm_runtime']
draw_table(df_ctx, col_ctx)

1_bench,2_bench,1_ctx,2_ctx,runtime,norm_runtime
parb_sad-0,nvd_conv-0,0.5,0.5,"[[], [79904, 60179, 39817, 54554, 51112, 36158, 51233, 49403, 35035, 44531, 24691, 9164], [222775, 257227]]","[[], [1.5475374276141227, 2.7917517164594545, 3.1092456660940186, 1.0565723471423314, 2.371126368528484, 2.8235202248945805, 0.9922530164817074, 2.2918444980515864, 2.7358269561143214, 0.8624523076327155, 1.1454351456670997, 0.7156020615336561], [1.1703686970044025, 1.2356939720604907]]"
parb_sad-0,nvd_conv-0,0.25,0.75,"[[], [120682, 65436, 37453, 88098, 53440, 44124, 74405, 24826, 9068], [224169, 250737]]","[[], [2.3373036623864585, 3.0356281313787346, 2.9246446977979073, 1.706234384986346, 2.479124141770273, 3.4455723879431517, 1.4410357716963957, 1.1516979031360177, 0.708105575511479], [1.177692202620491, 1.2045166311177726]]"
parb_sad-0,nvd_conv-0,0.75,0.25,"[[], [57535, 58037, 41430, 48948, 41217, 17930, 41243, 39681, 22359, 41004, 39197, 22264, 41731, 28251, 9066], [233726, 296690]]","[[], [1.1143067418124069, 2.6923826312859527, 3.2352022489458068, 0.9479983731334611, 1.9120894414548153, 1.400124941433703, 0.7987721031123506, 1.8408331786973464, 1.7459784476026863, 0.7941432804601708, 1.818380033401373, 1.7385600499765734, 0.8082234230046675, 1.310586379662275, 0.7079493987193503], [1.2279007701764155, 1.4252704598297496]]"
parb_sad-1,nvd_conv-0,0.5,0.5,"[[], [4815039, 3784634, 907171], [208799, 236771, 209355, 228803, 207294, 228246, 206345, 232239, 206359, 228128, 207729, 228746, 204581, 229212, 209619, 228169, 206378, 226733, 206028, 229401, 204865, 228913, 446775, 485604, 459607, 479317, 457192, 475391, 453561, 471411, 402902, 441135, 239744, 178062]]","[[], [1.6046353758082266, 1.6278205740289122, 1.6741642290976535], [1.0969445115736607, 1.1374252992832574, 1.099865508074769, 1.099147787321535, 1.0890378573755162, 1.0964720124517207, 1.0840521996784802, 1.115654003574105, 1.084125749950091, 1.0959051517073077, 1.0913231693862755, 1.0988739647585557, 1.0747848654555388, 1.1011125843085259, 1.1012524560537127, 1.0961021117964682, 1.0842255681758481, 1.089203704771238, 1.0823868113855821, 1.1020205222805095, 1.0762768852510691, 1.0996762168290386, 2.3471730427747364, 2.3327952960166023, 2.414587120296723, 2.302593147710459, 2.4018996984438865, 2.2837330181971907, 2.3828239101425823, 2.264613477834784, 2.1166822523194604, 2.1191704617513114, 1.2595168797873346, 0.855392863319306]]"
parb_sad-1,nvd_conv-0,0.25,0.75,"[[], [7749491, 3638755, 882465], [202694, 232685, 206124, 223150, 204791, 226408, 205894, 222317, 206189, 227482, 207217, 222947, 206305, 224576, 207279, 223934, 206650, 225307, 204890, 224163, 205903, 223824, 206071, 224222, 205646, 225442, 206160, 225503, 205242, 224183, 207350, 225487, 206430, 223286, 206456, 235325, 439520, 442850, 423767, 435910, 422453, 437138, 426656, 438107, 370630, 416983, 303195, 195664]]","[[], [2.5825559051769815, 1.565076108508927, 1.6285698467330423], [1.0648713395605898, 1.117796545031802, 1.082891156105198, 1.0719913145404585, 1.0758881195296985, 1.0876424357717953, 1.0816828302144517, 1.0679896619972713, 1.0832326395091045, 1.0928018293268769, 1.0886333308816576, 1.0710161219038834, 1.083842056045307, 1.078841682519552, 1.0889590535130762, 1.0757575757575757, 1.0856545448814265, 1.0823533367921447, 1.0764082250218023, 1.0768576699141061, 1.0817301125319156, 1.075229146250072, 1.0826127157912433, 1.0771411002863127, 1.0803799396887772, 1.08300186391499, 1.0830802853750539, 1.083294902096424, 1.0782574889937273, 1.0769537480063796, 1.0893320584619588, 1.0832180396226052, 1.0844987548989735, 1.0726446455679175, 1.084635348260536, 1.1304788532118906, 2.309058241307934, 2.1274091581637555, 2.226298424973469, 2.094070060144886, 2.2193952066237275, 2.0999692550104725, 2.241476048879409, 2.1046242385811187, 1.9471383690752628, 2.003146557521954, 1.5928624714992698, 0.9399511923291252]]"
parb_sad-1,nvd_conv-0,0.75,0.25,"[[], [3746237, 3352967, 825521], [224531, 270732, 211348, 257990, 209074, 2587

In [109]:
pair_series = df_ctx.iloc[3]
runtime = pair_series['runtime']
s1_runtime = runtime[1]
s2_runtime = runtime[2]

norm_runtime = pair_series['norm_runtime']
s1_norm = norm_runtime[1]
s2_norm = norm_runtime[2]

def get_from_to(duration):
    time_series = [0]

    for d in duration:
        new_elem = time_series[-1] + d
        time_series.append(new_elem)
        
    time_series = np.array(time_series)
    return time_series[0:-1], time_series[1:]

s1_from, s1_to = get_from_to(s1_runtime)
s1 = np.repeat("1", s1_from.shape[0])
k1 = np.arange(1, const.multi_kernel_app[pair_series['1_bench']]+1)
k1 = np.resize(k1, s1_from.shape[0])
k1 = ['{}:{}'.format('1', k) for k in k1]

s2_from, s2_to = get_from_to(s2_runtime)
s2 = np.repeat("2", s2_from.shape[0])
k2 = np.arange(1, const.multi_kernel_app[pair_series['2_bench']]+1)
k2 = np.resize(k2, s2_from.shape[0]).astype(str)
k2 = ['{}:{}'.format('2', k) for k in k2]

col_from = np.concatenate((s1_from, s2_from))
col_to = np.concatenate((s1_to, s2_to))
col_stream = np.concatenate((s1, s2))
col_kernel = np.concatenate((k1, k2))
col_norm = np.concatenate((s1_norm, s2_norm))



In [108]:
alt.renderers.enable('default')

data = pd.DataFrame()
data["start"] = col_from
data["end"] = col_to
data["stream"] = col_stream
data["kernel"] = col_kernel
data['position'] = (data['start'] + data['end']) / 2
data['norm'] = col_norm 
data['norm'] = data['norm'].round(2)


bars = alt.Chart(data).mark_bar().encode(
    x="start",
    x2="end",
    y="stream",
    color=alt.Color('kernel', scale=alt.Scale(scheme='dark2'))
).properties(
    width=900,
    height=100
)

text=alt.Chart(data).mark_text(
    align='center', 
    baseline='middle',
    color='white',
    fontSize=12
).encode(
    y=alt.Y('stream', title=None),
    x='position',
    text='norm'
)

alt.layer(bars, text, data=df)

alt.LayerChart(...)